In [1]:
import numpydoc as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
dataset=pd.read_csv('Social_Network_Ads.csv')
dataset

,User ID,Gender,Age,EstimatedSalary,Purchased
0,15624510,Male,19,19000,0
1,15810944,Male,35,20000,0
2,15668575,Female,26,43000,0
3,15603246,Female,27,57000,0
4,15804002,Male,19,76000,0
...,...,...,...,...,...
395,15691863,Female,46,41000,1
396,15706071,Male,51,23000,1
397,15654296,Female,50,20000,1
398,15755018,Male,36,33000,0


In [3]:
dataset=pd.get_dummies(dataset,dtype=int,drop_first=True)
dataset

,User ID,Age,EstimatedSalary,Purchased,Gender_Male
0,15624510,19,19000,0,1
1,15810944,35,20000,0,1
2,15668575,26,43000,0,0
3,15603246,27,57000,0,0
4,15804002,19,76000,0,1
...,...,...,...,...,...
395,15691863,46,41000,1,0
396,15706071,51,23000,1,1
397,15654296,50,20000,1,0
398,15755018,36,33000,0,1


In [4]:
dataset=dataset.drop("User ID",axis=1)
dataset

,Age,EstimatedSalary,Purchased,Gender_Male
0,19,19000,0,1
1,35,20000,0,1
2,26,43000,0,0
3,27,57000,0,0
4,19,76000,0,1
...,...,...,...,...
395,46,41000,1,0
396,51,23000,1,1
397,50,20000,1,0
398,36,33000,0,1


In [5]:
dataset['Purchased'].value_counts()

Purchased
0    257
1    143
Name: count, dtype: int64

In [6]:
dataset.columns

Index(['Age', 'EstimatedSalary', 'Purchased', 'Gender_Male'], dtype='object')

In [7]:
independent=dataset[['Age', 'EstimatedSalary','Gender_Male']]
dependent=dataset[['Purchased']]
dependent

,Purchased
0,0
1,0
2,0
3,0
4,0
...,...
395,1
396,1
397,1
398,0


In [8]:
independent.shape

(400, 3)

In [9]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(independent,dependent,test_size=1/3,random_state=0)

In [10]:
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
x_train=sc.fit_transform(x_train)
x_test=sc.transform(x_test)

In [12]:
from sklearn.ensemble import RandomForestClassifier

In [14]:
from sklearn.model_selection import GridSearchCV
param_grid={
    'criterion':['gini','entropy', 'log_loss'],
    'n_estimators': [50,100],
    'max_features':['sqrt', 'log2']
}
grid=GridSearchCV(RandomForestClassifier(),param_grid,refit=True,verbose=3,n_jobs=-1,scoring='f1_weighted')
grid.fit(x_train,y_train)

Fitting 5 folds for each of 12 candidates, totalling 60 fits


C:\Users\DELL\anaconda3\Lib\site-packages\sklearn\model_selection\_search.py:909: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.best_estimator_.fit(X, y, **fit_params)


GridSearchCV(estimator=RandomForestClassifier(), n_jobs=-1,
             param_grid={'criterion': ['gini', 'entropy', 'log_loss'],
                         'max_features': ['sqrt', 'log2'],
                         'n_estimators': [50, 100]},
             scoring='f1_weighted', verbose=3)

In [23]:
re=grid.cv_results_
grid_predictions=grid.predict(x_test)
from sklearn.metrics import confusion_matrix
cm=confusion_matrix(y_test,grid_predictions)
from sklearn.metrics import classification_report
clf_report=classification_report(y_test,grid_predictions)

In [25]:
print("the confusion Matrix:\n",cm)

the confusion Matrix:
 [[78  7]
 [ 4 45]]


In [27]:
print(clf_report)

              precision    recall  f1-score   support

           0       0.95      0.92      0.93        85
           1       0.87      0.92      0.89        49

    accuracy                           0.92       134
   macro avg       0.91      0.92      0.91       134
weighted avg       0.92      0.92      0.92       134



In [29]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test,grid.predict_proba(x_test)[:,1])

0.9637454981992797

In [31]:
table=pd.DataFrame.from_dict(re)
table

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_criterion,param_max_features,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.386716,0.010122,0.056605,0.010388,gini,sqrt,50,"{'criterion': 'gini', 'max_features': 'sqrt', ...",0.867478,0.889022,0.832483,0.925272,0.943699,0.891591,0.039828,5
1,0.733401,0.019649,0.068238,0.008611,gini,sqrt,100,"{'criterion': 'gini', 'max_features': 'sqrt', ...",0.826263,0.888107,0.851527,0.925272,0.962264,0.890687,0.049010,9
2,0.345141,0.012890,0.049053,0.008367,gini,log2,50,"{'criterion': 'gini', 'max_features': 'log2', ...",0.887346,0.868752,0.832483,0.925272,0.943699,0.891510,0.039707,7
3,0.704070,0.024192,0.078234,0.016684,gini,log2,100,"{'criterion': 'gini', 'max_features': 'log2', ...",0.887346,0.888107,0.851527,0.944161,0.943699,0.902968,0.035964,1
4,0.348951,0.009436,0.044335,0.002812,entropy,sqrt,50,"{'criterion': 'entropy', 'max_features': 'sqrt...",0.847141,0.870047,0.851527,0.925272,0.962264,0.891250,0.045108,8
5,0.636035,0.017291,0.054946,0.003447,entropy,sqrt,100,"{'criterion': 'entropy', 'max_features': 'sqrt...",0.847141,0.907177,0.851527,0.944161,0.943699,0.898741,0.042537,2
6,0.320159,0.010498,0.038643,0.001900,entropy,log2,50,"{'criterion': 'entropy', 'max_features': 'log2...",0.804764,0.888107,0.851527,0.925272,0.943699,0.882674,0.050216,12
7,0.628554,0.015101,0.055019,0.004554,entropy,log2,100,"{'criterion': 'entropy', 'max_features': 'log2...",0.847141,0.907177,0.851527,0.925272,0.943699,0.894963,0.039030,4
8,0.310580,0.012325,0.039705,0.002381,log_loss,sqrt,50,"{'criterion': 'log_loss', 'max_features': 'sqr...",0.847141,0.868752,0.851527,0.925272,0.943041,0.887147,0.039459,10
9,0.621811,0.020967,0.054571,0.005315,log_loss,sqrt,100,"{'criterion': 'log_loss', 'max_features': 'sqr...",0.826263,0.888107,0.851527,0.925272,0.943699,0.886974,0.043882,11


In [33]:
age_input=float(input("Age:"))
salary_input=float(input("EstimatedSalary:"))
sex_male_input=int(input("Sex Male 0 or 1:"))

Age: 67
EstimatedSalary: 140000
Sex Male 0 or 1: 0


In [35]:
grid.predict([[age_input,salary_input,sex_male_input]])

array([1], dtype=int64)

In [37]:
dir(classification_report)

['__annotations__',
 '__builtins__',
 '__call__',
 '__class__',
 '__closure__',
 '__code__',
 '__defaults__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__get__',
 '__getattribute__',
 '__getstate__',
 '__globals__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__kwdefaults__',
 '__le__',
 '__lt__',
 '__module__',
 '__name__',
 '__ne__',
 '__new__',
 '__qualname__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__']

In [39]:
import pickle
filename="finalized_RandomForestClassifier.sav"
pickle.dump(grid,open(filename,'wb'))

In [44]:
loaded_model=pickle.load(open("finalized_RandomForestClassifier.sav",'rb'))
result=loaded_model.predict([[43,90000,1]])
print(result)

[1]
